In [1]:
import pandas as pd

def load_some_data(path, separ = ','):
    return pd.read_csv(path, sep=separ, parse_dates = ['datetime'], index_col = 'datetime')

events_battle = load_some_data(r'data/events_battle.txt')
events_hero = load_some_data(r'data/events_hero.txt')
events_level = load_some_data(r'data/events_level.txt', '\t')
events_payment = pd.read_csv(r'data/events_payment.txt', parse_dates = ['created'], index_col = 'created')
events_quest = load_some_data(r'data/events_quest.txt')
events_resource = load_some_data(r'data/events_resource.txt')
events_user = load_some_data(r'data/events_user.txt')

In [208]:
#объявление всех сетов битв для анализа
no_bot = events_battle.loc[events_battle['opponent_type'] != 'bot']
duel = events_battle.loc[events_battle['kind'] =='duel']
only_bot = events_battle.loc[events_battle['opponent_type'] == 'bot']
duels_with_bots = events_battle.loc[(events_battle['opponent_type'] == 'bot') & (events_battle['kind'] =='duel')] #zero
duels_with_supers = duels_with_bots = events_battle.loc[(events_battle['opponent_race_id'] == 'super-hero') & (events_battle['kind'] =='duel')]#zero
donaters = events_payment.loc[events_payment['reason'] == 'market-buy'] #донатеры
#набор с челиками, которые создали одного перса - ниже

In [108]:
import numpy as np
def add_bool_col(df,label, users):
    tdf = df
    tdf = tdf.groupby('user_id').agg('count')
    tdf[label] = 1
    users[label] = tdf[label]
    users[label] = users[label].fillna(0)


In [41]:
counted_races = events_hero.groupby('user_id').agg('count').sort_values(by = 'race') #рас от 1 до 4, лол
#events_hero.loc[events_hero['race'] == 'super-hero'] # супергерой есть у каждого
def_users = events_user[~events_user.set_index('user_id').index.isin(counted_races.loc[counted_races['id'] > 1].index)]
def_users # пользователи только на суперменах
starter_battles = events_battle[events_battle.set_index('user_id').index.isin(def_users.set_index('user_id').index)]

In [119]:
def get_players_winrates(df):
    win = df.loc[df['status'] == 'win'][['user_id','status']].groupby('user_id').agg('count')
    tot = df[['user_id','status']].groupby('user_id').agg('count')
    return (win/tot).fillna(0)
def get_players_games(df):
    tot = df[['user_id','status']].groupby('user_id').agg('count')
    return tot.fillna(0)

In [128]:
def get_labels():
    used_cols = ['user_id']
    all_dates = events_battle[used_cols]
    all_dates = all_dates.append(events_resource[used_cols])
    all_dates = all_dates.append(events_quest[used_cols])
    all_dates = all_dates.append(events_payment[used_cols])
    last_dates = all_dates.reset_index().set_index(['user_id']).groupby(['user_id']).agg('max')
    last_dates.rename(index=str, columns={"index": "datetime"}, inplace = True)
    def is_leaved(date):
        current_date = pd.Timestamp(2018, 11, 8, 12)
        if (current_date - date).days > 3:
            return 1
        else:
            return 0
    last_dates['leaved'] = last_dates.apply(lambda dat : is_leaved(dat['datetime']),axis = 1)
    last_dates=last_dates[['leaved']]
    return last_dates

In [146]:
def get_quest_count():
    quest_count = events_battle[['user_id','id']].groupby('user_id').agg('count')
    quest_count['completed_quests'] = quest_count['id']
    quest_count = quest_count[['completed_quests']]
    return quest_count

,completed_quests
user_id,
00040422-b869-4272-ba05-1cc9481037c9,3
001a3472-44bc-4583-95d9-bf8759085017,86
002bcd7d-83b1-42ac-b257-ed71a4585f5e,125
0030c0b3-9683-41c0-bcac-b2aad03759f4,34
003c3099-4382-4f86-a989-a847628894f0,8
00467ba2-b71c-4a69-b8bc-d7491cf2eca0,3
0047e219-202a-4eda-8d31-b35a58f50eec,96
0049abe6-cc01-4c40-add7-ea480abcb0c4,3
0049d114-1c4f-4d9e-afce-bf2bc533eb80,4


In [163]:
# Среднее изменение количества ресурса в день
def get_average_resource_change(resource_name):
    changes = events_resource.loc[events_resource['res_id'] == resource_name]
    changes = changes.groupby(['user_id', changes.index.date]).agg('sum')[['total']]
    changes = changes.groupby('user_id').agg('mean')
    return changes

In [209]:
def get_most_race(ds):
    cur_race = pd.merge(ds, events_hero, on = 'hero_id')
    return cur_race.groupby("user_id_x")['race'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()



In [210]:
users = events_user[['user_id']]
users = users.set_index('user_id')
add_bool_col(duel, 'is_pvp', users)
add_bool_col(def_users, 'newbie', users)
users['donator'] = add_bool_col(donaters, 'donator', users)
users['pvp_efficiency'] = get_players_winrates(duel)['status'] * get_players_games(duel)['status']
users['pve_effifiency']  =get_players_winrates(only_bot)['status'] * get_players_games(only_bot)['status']
users['quest_completed'] = get_quest_count()['completed_quests']
users['gold_mean_changes'] = get_average_resource_change('gold')['total']
users['luck_mean_changes'] = get_average_resource_change('luck')['total']
users['fauvorite_battle_race'] = get_most_race(events_battle)['race']
users['fauvorite_quest_race'] = get_most_race(events_quest)['race']
users['leaved'] = get_labels()['leaved']
users['leaved'] = users[['leaved']].fillna(1)
users = users.fillna(0)

In [211]:
users.loc[users['donator'] == 1]

,is_pvp,newbie,donator,pvp_efficiency,pve_effifiency,quest_completed,gold_mean_changes,luck_mean_changes,fauvorite_battle_race,fauvorite_quest_race,leaved
user_id,,,,,,,,,,,


In [191]:
%matplotlib inline
from pandas.plotting import scatter_matrix
users.sort_values(by = ['pve_effifiency'], ascending = False)
#get_quest_count()

#users['quest_completed'].hist()
#scatter_matrix(users[['quest_completed', 'leaved']], figsize=(12, 8))
#опачки, почти все квестовоки играют
#Eusers.loc[(users['quest_completed'] < users['quest_completed'].median()) & (users['leaved'] == 0)].count() / users.loc[(users['quest_completed'] > users['quest_completed'].median())].count()

,is_pvp,newbie,pvp_efficiency,pve_effifiency,quest_completed,gold_mean_changes,food_mean_changes,fauvorite_battle_race,fauvorite_quest_race,leaved
user_id,,,,,,,,,,
359ead6f-c280-42b9-8512-3b8dae581faa,1.0,0.0,72.0,8215.0,8836.0,3153.250000,2.066667,elf,elf,0.0
4ad9a502-8326-443d-90cb-e4bdee947980,1.0,0.0,14.0,4775.0,4932.0,1286.653846,0.000000,orc,orc,0.0
667e3319-ca50-40e5-94c7-cfb5b2a15d0e,1.0,0.0,20.0,4432.0,4564.0,219.958333,0.000000,human,human,0.0
2b91bfae-6008-4bf8-bf45-9836d2edef15,1.0,0.0,48.0,4313.0,4576.0,105.026316,0.000000,human,human,0.0
3a69f8e6-5a0e-4f5e-a9d3-e8a4c3ad1b83,1.0,0.0,20.0,3886.0,4281.0,43.461538,0.000000,human,human,0.0
a27bc2ea-49ba-4c61-a32f-7319b9174e12,1.0,0.0,13.0,3646.0,3767.0,2348.733333,0.000000,elf,elf,0.0
8902d32d-fb45-4891-9a3a-5a0bb67943c0,1.0,0.0,7.0,2316.0,2365.0,6987.923077,0.538462,human,human,0.0
c3509c68-62cb-459e-949b-2274d825574a,1.0,0.0,10.0,2235.0,2318.0,23041.875000,0.333333,elf,elf,0.0
4989eaad-d456-4656-8b02-cc46e54f090e,1.0,0.0,9.0,2132.0,2349.0,6721.523810,0.714286,human,human,0.0


In [182]:

all_dates = events_battle[['user_id']]
all_dates = all_dates.append(events_resource[['user_id']])
all_dates = all_dates.append(events_quest[['user_id']])
all_dates = all_dates.append(events_payment[['user_id']])
all_dates.loc[all_dates['user_id'] == '33b8bddf-a67d-4ceb-a63d-0b077c711c96']

,user_id
